In [1]:
import time
import datetime
principio = datetime.datetime.now()
principio

datetime.datetime(2024, 1, 9, 20, 29, 8, 696231)

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.ets import AutoETS
from sktime.utils.plotting import plot_series
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.performance_metrics.forecasting import MeanSquaredError
from sklearn.model_selection import GridSearchCV
from sktime.transformations.series.boxcox import LogTransformer


# Read Data
data = pd.read_csv('../data/datos semanales.csv')
data.set_index('Fecha', inplace=True)
data['Billetes'] = data['Billetes'].astype('float64')
data.index = pd.to_datetime(data['Billetes'].index)
data['Billetes'].astype('float64').to_period('W-Sat')


Bad key savefig.frameon in file C:\Users\manue\anaconda3\envs\TecnicasPrediccion\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 421 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.4.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.level in file C:\Users\manue\anaconda3\envs\TecnicasPrediccion\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 472 ('verbose.level  : silent      # one of silent, helpful, debug, debug-annoying')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.4.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.fileo in file C:\Users\manue\anaconda3\envs\TecnicasPrediccion\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 473 ('verbose.fileo  : sys.stdout  # a log filename, sys.std

Fecha
2013-01-06/2013-01-12     62614.0
2013-01-13/2013-01-19     59301.0
2013-01-20/2013-01-26     56391.0
2013-01-27/2013-02-02     62981.0
2013-02-03/2013-02-09     65258.0
                           ...   
2016-11-27/2016-12-03    163841.0
2016-12-04/2016-12-10    127511.0
2016-12-11/2016-12-17    162260.0
2016-12-18/2016-12-24    135565.0
2016-12-25/2016-12-31    113289.0
Freq: W-SAT, Name: Billetes, Length: 208, dtype: float64

In [3]:
y_train, y_test = temporal_train_test_split(y = data['Billetes'].astype('float64').to_period('W'), test_size=5)

In [4]:
fh = np.arange(len(y_test)) + 1

In [5]:
from itertools import product
# Definir las opciones para cada parámetro
error_options = ["additive", "multiplicative"]
trend_options = ["additive", "multiplicative"]
seasonal_options = ["additive", "multiplicative"]
sp = [2, 3, 4, 5, 6, 7, 13, 52]

# Generar todas las combinaciones posibles
param_combinations = list(product(error_options, trend_options, seasonal_options, sp))

# Probar diferentes combinaciones
for params in param_combinations:
    autoETS_model = AutoETS(
        error=params[0],
        trend=params[1],
        seasonal=params[2],
        sp=params[3],
        n_jobs=-1
    )
    autoETS_model.fit(y_train)
    ETS_pred = autoETS_model.predict(fh)
    mape = mean_absolute_percentage_error(y_test, ETS_pred)
    print(f"Parameters: {params}, MAPE: {mape}")

Parameters: ('additive', 'additive', 'additive', 2), MAPE: 0.19397123710289804
Parameters: ('additive', 'additive', 'additive', 3), MAPE: 0.19369260878815925
Parameters: ('additive', 'additive', 'additive', 4), MAPE: 0.22310713415159428
Parameters: ('additive', 'additive', 'additive', 5), MAPE: 0.20524783626289259
Parameters: ('additive', 'additive', 'additive', 6), MAPE: 0.2241659869603871
Parameters: ('additive', 'additive', 'additive', 7), MAPE: 0.1665902181065831
Parameters: ('additive', 'additive', 'additive', 13), MAPE: 0.20828897723888
Parameters: ('additive', 'additive', 'additive', 52), MAPE: 0.15388724811176324
Parameters: ('additive', 'additive', 'multiplicative', 2), MAPE: 0.19956211640188304
Parameters: ('additive', 'additive', 'multiplicative', 3), MAPE: 0.20267447641479214
Parameters: ('additive', 'additive', 'multiplicative', 4), MAPE: 0.20944210404579797
Parameters: ('additive', 'additive', 'multiplicative', 5), MAPE: 0.2033254333898573
Parameters: ('additive', 'additi

C:\Users\manue\anaconda3\envs\TecnicasPrediccion\lib\site-packages\statsmodels\base\model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


Parameters: ('additive', 'additive', 'multiplicative', 52), MAPE: 0.14585495608305593
Parameters: ('additive', 'multiplicative', 'additive', 2), MAPE: 0.08958673699604407
Parameters: ('additive', 'multiplicative', 'additive', 3), MAPE: 0.08373052736709026
Parameters: ('additive', 'multiplicative', 'additive', 4), MAPE: 0.08212381044382028
Parameters: ('additive', 'multiplicative', 'additive', 5), MAPE: 0.0874282669509576
Parameters: ('additive', 'multiplicative', 'additive', 6), MAPE: 0.2061830512781418
Parameters: ('additive', 'multiplicative', 'additive', 7), MAPE: 0.26567463619452136
Parameters: ('additive', 'multiplicative', 'additive', 13), MAPE: 0.11372714408325246
Parameters: ('additive', 'multiplicative', 'additive', 52), MAPE: 0.8154017664313928
Parameters: ('additive', 'multiplicative', 'multiplicative', 2), MAPE: 0.08993697663583311
Parameters: ('additive', 'multiplicative', 'multiplicative', 3), MAPE: 0.08965839409304187
Parameters: ('additive', 'multiplicative', 'multiplic

C:\Users\manue\anaconda3\envs\TecnicasPrediccion\lib\site-packages\statsmodels\base\model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


Parameters: ('additive', 'multiplicative', 'multiplicative', 52), MAPE: 0.14763694249956658
Parameters: ('multiplicative', 'additive', 'additive', 2), MAPE: 0.3385908411458377
Parameters: ('multiplicative', 'additive', 'additive', 3), MAPE: 0.3293107118021201
Parameters: ('multiplicative', 'additive', 'additive', 4), MAPE: 0.2675933234180521
Parameters: ('multiplicative', 'additive', 'additive', 5), MAPE: 0.3386949605749637
Parameters: ('multiplicative', 'additive', 'additive', 6), MAPE: 0.340477946566257
Parameters: ('multiplicative', 'additive', 'additive', 7), MAPE: 0.37267460642021055
Parameters: ('multiplicative', 'additive', 'additive', 13), MAPE: 0.29863643085977876
Parameters: ('multiplicative', 'additive', 'additive', 52), MAPE: 0.17761833163470736
Parameters: ('multiplicative', 'additive', 'multiplicative', 2), MAPE: 0.3309151646178862
Parameters: ('multiplicative', 'additive', 'multiplicative', 3), MAPE: 0.31143425024333254
Parameters: ('multiplicative', 'additive', 'multipl

...

Parameters: ('multiplicative', 'additive', 'multiplicative', 5), MAPE: 3467.758089338582
Parameters: ('multiplicative', 'additive', 'multiplicative', 6), MAPE: 0.24934354694804278
Parameters: ('multiplicative', 'additive', 'multiplicative', 7), MAPE: 0.3244749327186917
Parameters: ('multiplicative', 'additive', 'multiplicative', 13), MAPE: 0.39959691886225857


C:\Users\manue\anaconda3\envs\TecnicasPrediccion\lib\site-packages\statsmodels\base\model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


Parameters: ('multiplicative', 'additive', 'multiplicative', 52), MAPE: 0.2808377851167556
Parameters: ('multiplicative', 'multiplicative', 'additive', 2), MAPE: 0.08965335287566845
Parameters: ('multiplicative', 'multiplicative', 'additive', 3), MAPE: 0.08357194627529499
Parameters: ('multiplicative', 'multiplicative', 'additive', 4), MAPE: 0.11071309391830633
Parameters: ('multiplicative', 'multiplicative', 'additive', 5), MAPE: 0.08658819047996164
Parameters: ('multiplicative', 'multiplicative', 'additive', 6), MAPE: 0.08605884107003724
Parameters: ('multiplicative', 'multiplicative', 'additive', 7), MAPE: 0.08371251412252592
Parameters: ('multiplicative', 'multiplicative', 'additive', 13), MAPE: 0.11540459717418203
Parameters: ('multiplicative', 'multiplicative', 'additive', 52), MAPE: 1786458.9035548414
Parameters: ('multiplicative', 'multiplicative', 'multiplicative', 2), MAPE: 0.09044331566444419
Parameters: ('multiplicative', 'multiplicative', 'multiplicative', 3), MAPE: 0.0812

C:\Users\manue\anaconda3\envs\TecnicasPrediccion\lib\site-packages\statsmodels\base\model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


Parameters: ('multiplicative', 'multiplicative', 'multiplicative', 52), MAPE: 0.08735946053320053


El mejor: Parameters: ('additive', 'multiplicative', 'additive', 4), MAPE: 0.08212381044382028

In [6]:
data_auto_model = AutoETS(
        error="additive",
        trend="multiplicative",
        seasonal="additive",
        sp=4,
        n_jobs=1)

In [7]:
data_auto_model.fit(y_train)

AutoETS(error='additive', n_jobs=1, seasonal='additive', sp=4,
        trend='multiplicative')

In [8]:
y_pred = data_auto_model.predict(fh)

In [9]:
mape = mean_absolute_percentage_error(y_test[0:7], y_pred[0:7])
mse = MeanSquaredError()
mse = mse(y_test[0:7], y_pred[0:7])
r2 = r2_score(list(y_test[0:7]), list(y_pred[0:7]))

print(f'MAPE: {mape}')
print(f'MSE: {mse}')
print(f'R2 Score: {r2}')


MAPE: 0.08212381044382028
MSE: 189710073.75311643
R2 Score: 0.5140141399257458


In [10]:
final = datetime.datetime.now()
duracion = final-principio
print(principio)
print(final)
print(duracion)

2024-01-09 20:29:08.696231
2024-01-09 20:29:27.741458
0:00:19.045227
